In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from astropy.coordinates import SkyCoord, AltAz, EarthLocation
from astropy.time import Time
from astropy import units as u

In [2]:
file = "data/exoplanet.eu_catalog.csv"

In [3]:
data = pd.read_csv(file, skiprows=0, usecols=['radius', 'orbital_period', 'ra', 'dec', 'mag_v', 
                                              'star_distance', 'star_radius', 'semi_major_axis',
                                              'eccentricity', 'tzero_tr'])

In [4]:
# We want to drop rows with null planet and stellar radii
# We also need the V-band magnitudes
data = data.dropna(subset={"radius", "star_radius", "mag_v", "tzero_tr"})
# We want only stars with mag_v ~< 12
data = data[data["mag_v"] < 12]

# Now compute transit depth for each
transit_depth = (0.10049*data["radius"] / data["star_radius"])**2
# Compute transit signature in magnitudes
transit_sig = -2.5 * np.log10(1 - transit_depth)
data["transit_sig"] = transit_sig
# Drop all rows where transit signature is > 0.008
data = data[transit_sig > 0.008]

In [5]:
semi_minor_axis = data["semi_major_axis"] * np.sqrt(1 - data["eccentricity"]**2)
# This is the lower bound on circumference
circ = np.sqrt(2) * np.pi * np.sqrt(data["semi_major_axis"]**2 + semi_minor_axis**2)
# Velocity in meters per hour
veloc = circ / data["orbital_period"] * 1.496e11 / 24
# Very rough duration estimate
transit_dur = 2 * data["star_radius"] * 6.957e8 / veloc
data["transit_dur"] = transit_dur
# Keep transit duration under 3 hours
data = data[transit_dur < 3]

In [6]:
def from_jd(jd):
    
    A = jd // 1
    L = A+68569
    N = 4*L//146097
    L = L-(146097*N+3)//4
    I = 4000*(L+1)//1461001
    L = L-1461*I//4+31
    J = 80*L//2447
    K = L-2447*J//80
    L = J//11
    J = J+2-12*L
    I = 100*(N-49)+I+L
    
    B = jd % 1
    hour = B * 24
    minute = (hour % 1) * 60
    second = (minute % 1) * 60
    
    I = map(int, I)
    J = map(int, J)
    K = map(int, K)
    hour = map(int, hour)
    minute = map(int, minute)
    second = map(int, second)
    
    return [datetime(i, j, k, h, m, s) for i, j, k, h, m, s
            in zip(I, J, K, hour, minute, second)]

def equinox(year):
    
    return datetime(year, 3, 21, 0, 0, 0)

def delta_ra(dates):
    
    dts = (dateobj - equinox(dateobj.year) for dateobj in dates)
    return np.array([dt.days for dt in dts])

In [7]:
# September 4th at 21:00 EST
after = 2458747.583
# Compute the next transit
rem = (after - data["tzero_tr"]) % data["orbital_period"]
obsday = after - rem + data["orbital_period"]

for i in obsday.index:
    
    while not (0.583 <= obsday[i] % 1 <= 0.708): obsday[i] += data["orbital_period"][i]
    
data['obsday'] = obsday

In [8]:
at_stony = EarthLocation(lat=40.914224*u.deg, lon=-73.11623*u.deg, height=0)
skycoords = SkyCoord(data['ra']*u.deg, data['dec']*u.deg, frame='icrs')
times = Time(data['obsday'], format='jd')
altaz = AltAz(location=at_stony, obstime=times)
aacoords = skycoords.transform_to(altaz)

data["alt"] = aacoords.alt
data["az"] = aacoords.az
data = data[data["alt"] > 40]

In [9]:
# data = data[data["dec"] > -19]
# dra = delta_ra(from_jd(data["tzero_tr"]))
# ra = (dra * 360 / 365.25 + 180 + 75) % 360
# data["ra"] = ra
# data = data[ra > 285]

In [10]:
data['obsday'] -= 5 / 24

for i, date in zip(data["obsday"].index, from_jd(data["obsday"])):
    
    print(i, date)

320 2019-09-28 11:58:22
695 2019-09-20 10:25:29
3826 2019-09-23 10:00:38
3946 2019-10-01 11:51:40


In [11]:
data[['ra', 'dec', 'transit_sig', 'transit_dur', 'alt', 'az']]

,ra,dec,transit_sig,transit_dur,alt,az
320,9.575000,42.463056,0.011971,2.954782,88.263412,17.227400
695,300.179167,22.710833,0.022135,2.019375,53.672104,251.693643
3826,286.808333,49.316389,0.015612,2.468977,55.872362,300.722504
3946,3.962500,1.200556,0.012462,2.721241,49.999685,189.681904
